### Reading Training Data

##### Authorize Earth Engine

In [1]:
import ee

ee.Authenticate()

In [2]:
!earthengine set_project warm-dynamics-433216-k3

Successfully saved project id


#### Get masks from all over the world

In [3]:
import ee
import folium

# Define a function to add Earth Engine layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds Earth Engine image tiles to a folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom method to folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Initialize the Earth Engine API
ee.Initialize()

# Load the dataset
dataset = ee.Image('USGS/GFSAD1000_V1')
crop_mask = dataset.select('landcover')

# Define visualization parameters
crop_mask_vis = {
    'min': 0,
    'max': 5,
    'palette': ['black', 'orange', 'brown', '02a50f', 'green', 'yellow'],
}

# Create a map centered on the desired coordinates
map = folium.Map(location=[13.72, -17.22], zoom_start=2)

# Add the crop mask layer to the map
map.add_ee_layer(crop_mask, crop_mask_vis, 'Crop Mask')

# Add layer control to toggle layers on and off
map.add_child(folium.LayerControl())

# Display the map
map

### Reading Test Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install rasterio
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 50.1 MB/s eta 0:00:00


#### How masks look over Azerbaijan (2020-06-08)

In [6]:
import rasterio

# Path to your .TIF file
tif_paths = ["/content/drive/MyDrive/Colab Notebooks/EOC_Data/Agriculture/DIM_SPOT6_PMS_202006080723476_ORT_AKWOI-00082830/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/IMG_SP~1.TIF",
             "/content/drive/MyDrive/Colab Notebooks/EOC_Data/Agriculture/DIM_SPOT6_PMS_202006080723476_ORT_AKWOI-00082830/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/IMG_SP~2.TIF",
             "/content/drive/MyDrive/Colab Notebooks/EOC_Data/Agriculture/DIM_SPOT6_PMS_202006080723476_ORT_AKWOI-00082830/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/IMG_SP~3.TIF",
             "/content/drive/MyDrive/Colab Notebooks/EOC_Data/Agriculture/DIM_SPOT6_PMS_202006080723476_ORT_AKWOI-00082830/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/IMG_SP~4.TIF"]

# Open the .TIF file
for tif_path in tif_paths:
  with rasterio.open(tif_path) as dataset:
      # Get the coordinates of the upper-left corner
      ul_x, ul_y = dataset.transform * (0, 0)
      print(f"Upper-left corner coordinates: ({ul_x}, {ul_y})")

      # Get the coordinates of the lower-right corner
      lr_x, lr_y = dataset.transform * (dataset.width, dataset.height)
      print(f"Lower-right corner coordinates: ({lr_x}, {lr_y})")

      # Print the bounds of the image
      print(f"Bounds of the image: {dataset.bounds}")

      # Extracting the coordinates of each corner
      corners = {
          "upper_left": (ul_x, ul_y),
          "lower_right": (lr_x, lr_y),
          "upper_right": (dataset.bounds.right, dataset.bounds.top),
          "lower_left": (dataset.bounds.left, dataset.bounds.bottom)
      }

      for corner, coords in corners.items():
          print(f"{corner}: {coords}")

Upper-left corner coordinates: (48.25284722222222, 40.06994444444445)
Lower-right corner coordinates: (48.480402777777776, 39.8423888888889)
Bounds of the image: BoundingBox(left=48.25284722222222, bottom=39.8423888888889, right=48.480402777777776, top=40.06994444444445)
upper_left: (48.25284722222222, 40.06994444444445)
lower_right: (48.480402777777776, 39.8423888888889)
upper_right: (48.480402777777776, 40.06994444444445)
lower_left: (48.25284722222222, 39.8423888888889)
Upper-left corner coordinates: (48.48040277777778, 40.06994444444445)
Lower-right corner coordinates: (48.66894444444445, 39.8423888888889)
Bounds of the image: BoundingBox(left=48.48040277777778, bottom=39.8423888888889, right=48.66894444444445, top=40.06994444444445)
upper_left: (48.48040277777778, 40.06994444444445)
lower_right: (48.66894444444445, 39.8423888888889)
upper_right: (48.66894444444445, 40.06994444444445)
lower_left: (48.48040277777778, 39.8423888888889)
Upper-left corner coordinates: (48.2528472222222

Finding borders:

In [7]:
# Coordinates from the four files
coordinates = {
    "upper_left_1": (48.25284722222222, 40.06994444444445),
    "lower_right_1": (48.480402777777776, 39.8423888888889),
    "upper_left_2": (48.48040277777778, 40.06994444444445),
    "lower_right_2": (48.66894444444445, 39.8423888888889),
    "upper_left_3": (48.25284722222222, 39.84238888888889),
    "lower_right_3": (48.480402777777776, 39.80480555555556),
    "upper_left_4": (48.48040277777778, 39.84238888888889),
    "lower_right_4": (48.66894444444445, 39.80480555555556)
}

# Calculating the combined bounding box
min_longitude = min(coordinates["upper_left_1"][0], coordinates["upper_left_3"][0], coordinates["lower_right_1"][0], coordinates["lower_right_3"][0])
max_longitude = max(coordinates["upper_left_2"][0], coordinates["upper_left_4"][0], coordinates["lower_right_2"][0], coordinates["lower_right_4"][0])
min_latitude = min(coordinates["lower_right_1"][1], coordinates["lower_right_2"][1], coordinates["lower_right_3"][1], coordinates["lower_right_4"][1])
max_latitude = max(coordinates["upper_left_1"][1], coordinates["upper_left_2"][1], coordinates["upper_left_3"][1], coordinates["upper_left_4"][1])

combined_bbox = (min_longitude, min_latitude, max_longitude, max_latitude)
print(f"Combined Bounding Box: {combined_bbox}")

Combined Bounding Box: (48.25284722222222, 39.80480555555556, 48.66894444444445, 40.06994444444445)


In [8]:
# Define the combined bounding box as an Earth Engine Geometry
combined_bbox = ee.Geometry.Rectangle([48.25284722222222, 39.80480555555556, 48.66894444444445, 40.06994444444445])

# Extract the crop mask for the entire bounding box
crop_mask = ee.Image('USGS/GFSAD1000_V1').clip(combined_bbox)

# Define visualization parameters if needed
crop_mask_vis = {
    'min': 0,
    'max': 5,
    'palette': ['black', 'orange', 'brown', '02a50f', 'green', 'yellow'],
}

# Display or export the combined data
# Example: Displaying on a map (requires folium or other visualization tools)
map = folium.Map(location=[(39.80480555555556 + 40.06994444444445) / 2, (48.25284722222222 + 48.66894444444445) / 2], zoom_start=12)
map.add_ee_layer(crop_mask, crop_mask_vis, 'Combined Crop Mask')
map.add_child(folium.LayerControl())
display(map)

- **Black:** Non-croplands (value 0)
- **Orange:** Croplands: irrigation major (value 1)
- **Brown:** Croplands: irrigation minor (value 2)
- **Dark Green (#02a50f):** Croplands: rainfed (value 3)
- **Green: Croplands:** rainfed, minor fragments (value 4)
- **Yellow:** Croplands: rainfed, very minor fragments (value 5)